In [3]:
# NIN 网络中的网络，串联多个由卷积层和全连接层构成的小网络来构建一个深层网络
# AlexNet和VGG在网络的宽度（通道数）和深度层次上扩展了LeNet，NIN则彻底分成整个分类块
# 的层次上进行拓展LeNet

# NiN块 使用三层卷积层但是最后不再使用全连接层，，最后一层卷积层的输出通道数
# 设置成分类的个数，并且使用全局平均池化操作作为输出进行分类
import time
import torch
from torch import nn,optim
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def nin_block(in_channels, out_channels, kernel_size, stride, padding):
    blk = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU(),
                        nn.Conv2d(out_channels, out_channels, kernel_size=1),
                        nn.ReLU())
    return blk

In [4]:
import torch.nn.functional as F
class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d,self).__init__()
        
    def forward(self,X):
        return F.avg_pool2d(X,kernel_size = X.size()[2:])
    
net = nn.Sequential(
    nin_block(1,96,kernel_size = 11,stride=4,padding = 0),
    nn.MaxPool2d(kernel_size=3,stride=2),
    
    nin_block(96,256,kernel_size = 5,stride=1,padding = 2),
    nn.MaxPool2d(kernel_size=3,stride=2),
    
    nin_block(256,384,kernel_size = 3,stride=1,padding = 1),
    nn.MaxPool2d(kernel_size=3,stride=2),
    
    nn.Dropout(0.5),
    nin_block(384,10,kernel_size=3,stride=1,padding=0),
    GlobalAvgPool2d(),
    d2l.FlattenLayer()
)

X = torch.rand(1, 1, 224, 224)
for name, blk in net.named_children(): 
    X = blk(X)
    print(name, 'output shape: ', X.shape)

0 output shape:  torch.Size([1, 96, 54, 54])
1 output shape:  torch.Size([1, 96, 26, 26])
2 output shape:  torch.Size([1, 256, 26, 26])
3 output shape:  torch.Size([1, 256, 12, 12])
4 output shape:  torch.Size([1, 384, 12, 12])
5 output shape:  torch.Size([1, 384, 5, 5])
6 output shape:  torch.Size([1, 384, 5, 5])
7 output shape:  torch.Size([1, 10, 3, 3])
8 output shape:  torch.Size([1, 10, 1, 1])
9 output shape:  torch.Size([1, 10])


In [ ]:
# 获取数据和训练模型
batch_size = 128
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size,resize=224)

lr,num_epochs = 0.002,5
optimizer = torch.optim.Adam(net.parameters(),lr = lr)
d2l.train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)


training on  cpu


In [ ]:
# 这里总结一下NIN与vgg的主要区别
# NIN与vgg都使用卷积块的思想，但是vgg是完整的卷积块，每个卷积块
# 跟一个池化层，而NIN块中没有池化层，而是在所有卷积块之后统一使用一个池化层代替vgg中的
# 最后的全连接层，进行直接分类。
#